In [ ]:
!gdown 1mrBIEiHWKnBlOAyjvDtGEBpG5qGDv_o4


Downloading...
From (original): https://drive.google.com/uc?id=1mrBIEiHWKnBlOAyjvDtGEBpG5qGDv_o4
From (redirected): https://drive.google.com/uc?id=1mrBIEiHWKnBlOAyjvDtGEBpG5qGDv_o4&confirm=t&uuid=01250e16-a8c7-4c99-b946-138291629d5c
To: /content/RAW_recipes.csv
100% 295M/295M [00:06<00:00, 45.8MB/s]


In [ ]:
import tarfile
import os
import re
import torch
import random
import numpy as np
import pandas as pd
from io import StringIO
from sentence_transformers import SentenceTransformer, CrossEncoder


In [ ]:
data_path = "RAW_recipes.csv"
raw_df = pd.read_csv(data_path)
raw_df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


## Data cleaning

merge column name tags ingredients into column text

In [ ]:
# drop unused column and save to pickle (for recipe details api purposes)
clean_df = raw_df.drop(columns=['contributor_id', 'submitted', 'nutrition'])
clean_df.to_pickle('clean_recipe.pkl')
recipes_df = pd.read_pickle("clean_recipe.pkl")
# recipes_df


## preprocess for pyterrier compatibilty


In [ ]:
# preprocess for pyterrier compatibilty
def merge_column(row):
    name = row['name'] if pd.notnull(row['name']) else ''
    tags = ' '.join(eval(row['tags'])) if pd.notnull(row['tags']) else ''
    ingredients = ' '.join(eval(row['ingredients'])) if pd.notnull(row['ingredients']) else ''
    return f"{name} {tags} {ingredients}"


In [ ]:
# Untuk preprocess text_raw menjadi text
def remove_nonalphanum(text):
  pattern = re.compile('[\W_]+')
  return pattern.sub(' ', text)

def preprocess_text(text):
    """
    Preprocessing steps dengan justifikasi:
    1. Lowercase - menyeragamkan teks untuk menghindari mismatch karena perbedaan kapitalisasi
    2. Remove special characters - membersihkan noise dan karakter yang tidak bermakna
    3. Remove excessive whitespace - normalisasi format teks
    """
    # lowercase
    text = text.lower()
    # remove non-alphanumeric characters
    text = remove_nonalphanum(text)
    # remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
docs_df = recipes_df.copy()
docs_df['text_raw'] = docs_df.apply(merge_column,axis=1)
# docs_df = docs_df[['recipe_id', 'text']]
# docs_df = docs_df.rename(columns={'recipe_id': 'docno'})
docs_df = docs_df[['id','text_raw']]


In [ ]:
# datatypes = docs_df.dtypes
# value = docs_df.loc[0, 'text_raw']
# re.sub(r'\s+', ' ', value)
# steps = docs_df.loc[0, 'steps']
# eval(steps)
docs_df = docs_df.rename(columns={'id': 'docno'})
docs_df['docno'] = docs_df['docno'].astype(str)
docs_df['text'] = docs_df['text_raw'].apply(preprocess_text)
docs_df


,docno,text_raw,text
0,137739,arriba baked winter squash mexican style 60-...,arriba baked winter squash mexican style 60 mi...
1,31490,a bit different breakfast pizza 30-minutes-or...,a bit different breakfast pizza 30 minutes or ...
2,112140,all in the kitchen chili time-to-make course ...,all in the kitchen chili time to make course p...
3,59389,alouette potatoes 60-minutes-or-less time-to-...,alouette potatoes 60 minutes or less time to m...
4,44061,amish tomato ketchup for canning weeknight t...,amish tomato ketchup for canning weeknight tim...
...,...,...,...
231632,486161,zydeco soup ham 60-minutes-or-less time-to-mak...,zydeco soup ham 60 minutes or less time to mak...
231633,493372,zydeco spice mix 15-minutes-or-less time-to-ma...,zydeco spice mix 15 minutes or less time to ma...
231634,308080,zydeco ya ya deviled eggs 60-minutes-or-less t...,zydeco ya ya deviled eggs 60 minutes or less t...
231635,298512,cookies by design cookies on a stick 30-minu...,cookies by design cookies on a stick 30 minute...


In [ ]:
# save docs_df
docs_df.to_pickle('docs_df_preprocessed.pkl')


# Indexing

In [ ]:
!pip install python-terrier -q
!pip install -U sentence-transformers -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from sentence_transformers.util import cos_sim
import pyterrier as pt
from pyterrier.measures import *
if not pt.java.started():
    pt.java.init()

terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


In [ ]:
# Find the maximum length of the 'text_raw' column
max_text_raw_length = docs_df['text_raw'].str.len().max()
print(f"maximum length of the 'text_raw' column is: {max_text_raw_length}")
max_docno_length = docs_df['docno'].str.len().max()
print(f"maximum length of the 'docno' column is: {max_docno_length}")

maximum length of the 'text_raw' column is: 911
maximum length of the 'docno' column is: 6


In [ ]:
# not removing stopwords maybe to gain more context
indexer = pt.IterDictIndexer('./recipe_index',\
                          type = pt.index.IndexingType(1), \
                          tokeniser = pt.index.TerrierTokeniser('utf'), \
                          stemmer = pt.index.TerrierStemmer('porter'), \
                          stopwords = None, \
                          blocks = True, verbose = True, meta={'text_raw':950,'docno':10},overwrite=True)
indexref = indexer.index(docs_df.to_dict(orient='records'))
index = pt.IndexFactory.of(indexref)

In [ ]:
# load index
indexref_test = pt.IndexRef.of("./recipe_index")
bm25 = pt.terrier.Retriever(indexref_test, wmodel="BM25")


In [ ]:
temp_pipeline = (bm25 % 50)
temp_pipeline.search('tomato with peanut butter cookies')

,qid,docid,docno,rank,score,query
0,1,155882,335741,0,19.128109,tomato with peanut butter cookies
1,1,155937,407855,1,18.853554,tomato with peanut butter cookies
2,1,116650,473389,2,18.785295,tomato with peanut butter cookies
3,1,156295,128122,3,18.784233,tomato with peanut butter cookies
4,1,88169,223013,4,18.765799,tomato with peanut butter cookies
5,1,156280,51911,5,18.759707,tomato with peanut butter cookies
6,1,156234,396857,6,18.620690,tomato with peanut butter cookies
7,1,55392,109838,7,18.506496,tomato with peanut butter cookies
8,1,155932,225467,8,18.496298,tomato with peanut butter cookies
9,1,52169,460749,9,18.360580,tomato with peanut butter cookies


## Reranker

retrieve top 50 using bm25, rerank using crossencoder based on raw_text and query pair.

In [ ]:
crossmodel = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Training Reranker


In [ ]:
# Train a pretrained reranker model

from sentence_transformers import InputExample, CrossEncoder
from torch.utils.data import DataLoader
import random
import pandas as pd

def create_training_data(docs_df, bm25_pipeline, num_samples=1000):
    training_data = []
    for _ in range(num_samples):
        query = random.choice(["chocolate cake", "vegan pasta", "quick breakfast", "low-carb snacks", "banana ice cream", "fruit milkshake"])

        results = bm25_pipeline.search(query)

        results_with_text = pd.merge(results, docs_df, on='docno', how='left')

        if len(results) > 1:
            positive = results_with_text.iloc[0]  # Top result
            negative = results_with_text.iloc[1]  # Second result

            training_data.append(InputExample(texts=[query, positive['text_raw']], label=1))

            training_data.append(InputExample(texts=[query, negative['text_raw']], label=0))

    return training_data

training_data = create_training_data(docs_df, temp_pipeline, num_samples=1000)

train_dataloader = DataLoader(training_data, shuffle=True, batch_size=16)

crossmodel.fit(
    train_dataloader=train_dataloader,
    epochs=3,
    warmup_steps=100,
    output_path="crossencoder_model"
)

crossmodel.save("crossencoder_model")
# crossmodel.save('crossencoder_model')

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
crossmodel = CrossEncoder('crossencoder_model')


In [ ]:
import pickle

# Save the CrossEncoder model
with open('crossencoder_model.pkl', 'wb') as f:
    pickle.dump(crossmodel, f)

print("Model saved as 'crossencoder_model.pkl'")


Model saved as 'crossencoder_model.pkl'


In [ ]:
def _crossencoder_apply(df: pd.DataFrame):
    # Score pairs of (query, text) using the CrossEncoder
    return crossmodel.predict(list(zip(df['query'].values, df['text_raw'].values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=128)

In [ ]:
bm25_reranker_pipeline = (
    bm25 % 50 >>
    pt.text.get_text(index, ['text_raw']) >>
    cross_encT % 30
)

In [ ]:
results1 = bm25_reranker_pipeline.search('peanut buter cookies')
results2 = bm25_reranker_pipeline.search('paenut butter cookies')
docnos1 = results1['docno'].tolist()
docnos2 = results2['docno'].tolist()
common_docnos = list(set(docnos1) & set(docnos2))
print(f"Common docno values: {common_docnos}")

Common docno values: ['210269', '307836', '75086', '145504', '216619', '213906', '507289', '322997', '380077', '287523', '215880']


In [ ]:
recipes_df = pd.read_pickle("clean_recipe.pkl")
def search_recipes(query):
    """
    Searches the recipes_df DataFrame for recipes matching the given query,
    including the rank from the search results.
    """
    # Preprocess the query
    query = preprocess_text(query)

    # Search using bm25_reranker_pipeline
    results = bm25_reranker_pipeline.search(query)
    results['docno'] = results['docno'].astype(np.int64)


    # Merge search results with recipes_df
    matching_recipes = pd.merge(results, recipes_df, left_on='docno', right_on='id', how='left')

    #
    matching_recipes = matching_recipes[['rank'] + recipes_df.columns.tolist()]

    return matching_recipes

In [ ]:
# bm25_reranker_pipeline.search('paenut butter cookies')

In [ ]:
search_recipes('paenut butter cookies under 30 minutes')

,rank,name,id,minutes,tags,n_steps,steps,description,ingredients,n_ingredients
0,0,evil peanut butter cookies,115689,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat the oven to 350', 'in a large bowl ,...",these are really wicked cookies!,"['flour', 'sugar', 'brown sugar', 'baking powd...",11
1,1,peanut butter devil cookies,307836,29,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 350', 'in a large bowl , com...",i got this off the side of a devil's food cake...,"[""devil's food cake mix"", 'vegetable oil', 'eg...",6
2,2,butter pecan cookies,305119,25,"['30-minutes-or-less', 'time-to-make', 'course...",7,"['cream butter and sugar', 'add egg', 'add flo...",simple basic cookie that most everyone loves. ...,"['butter', 'egg', 'baking powder', 'pecans', '...",8
3,3,best soft peanut butter cookies,216619,20,"['30-minutes-or-less', 'time-to-make', 'course...",13,"['preheat oven to 325 degrees', 'combine flour...",i made this recipe when i was craving peanut b...,"['flour', 'baking soda', 'butter', 'granulated...",8
4,4,moist peanut butter cookies,213906,17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 375', 'mix all ingredients t...",i love these cookies since they are moist and ...,"['butter', 'peanut butter', 'brown sugar', 'gr...",10
5,5,michelle s peanut butter chocolate chip cookies,285626,25,"['30-minutes-or-less', 'time-to-make', 'course...",7,"['pre-heat oven to 350', 'cream butter , peanu...",delicious peanut butter cookies that i added c...,"['butter', 'peanut butter', 'sugar', 'brown su...",10
6,6,peanut butter cookies,145504,30,"['30-minutes-or-less', 'time-to-make', 'course...",7,"['heat oven to 375', 'beat peanut butter , wat...",these are the easiest peanut butter cookies yo...,"['crunchy peanut butter', 'water', 'eggs', 'ye...",4
7,7,oatmeal peanut butter cookies sugar free,322997,23,"['30-minutes-or-less', 'time-to-make', 'course...",4,"['mix all together', 'roll into small balls an...",i have not tried this recipe. i got this recip...,"['natural-style peanut butter', 'oatmeal', 'sp...",5
8,8,peanut butter ball cookies,75086,22,"['30-minutes-or-less', 'time-to-make', 'course...",11,"['set oven to 375 degrees', 'combine sugar , p...","this is a super peanut butter cookie recipe, i...","['sugar', 'peanut butter', 'butter', 'egg whit...",8
9,9,homemade peanut butter cookies,308257,17,"['30-minutes-or-less', 'time-to-make', 'course...",6,"['preheat oven to 350f', 'mix all ingredients ...",my grandma passed this recipe down to me when ...,"['peanut butter', 'sugar', 'egg', 'vanilla']",4
